## Credits-
### - @richolson for vLLM 0.6.3 utility script
### - @takanashihumbert for Qwen2.5-32B-awq model

In [ ]:
import pandas as pd
import numpy as np
import os

### change MODEL PATH here
os.environ['llm_path'] = '/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1'

os.environ["TOKENIZERS_PARALLELISM"] = "false"

IS_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

In [ ]:
df = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')
df.head(3)

In [ ]:
%%writefile run_vllm.py

import sys
import re
import gc
import vllm
print('vllm version=',vllm.__version__)
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

df = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')

llm = vllm.LLM(model=os.getenv("llm_path"),
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.98,
          max_model_len=1024,
          tensor_parallel_size=2,
          trust_remote_code=True)
tokenizer = llm.get_tokenizer()

def apply_template(topic, tokenizer):
    messages = [
        {"role": "system", 
         "content": '''You are an expert essay writer. Write a comprehensive essay on the given topic.
IMPORTANT: Limit the essay to approximately 100-150 words.'''
        },
        {
            "role": "user", 
            "content": topic
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

df["topic"] = df['topic'].apply(lambda x: apply_template(x, tokenizer))
print('Example input-\n',df["topic"][0])

responses = llm.generate(
    df["topic"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_p=0.9,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0.7,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=199,  # Maximum number of tokens to generate per output sequence.
    ),
    use_tqdm = True
)

df["essay"] = [x.outputs[0].text for x in responses]
df.to_csv("submission.csv", columns=["id", "essay"], index=False)

In [ ]:
!python run_vllm.py

In [ ]:
df = pd.read_csv('submission.csv')
df.head(3)